In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [4]:
def find_points1(omega1, omega2, n_peaks, limit):
    counter = 0
    in_peak = False
    same_time = []
    index2 = 0 

    for i in range(len(omega1)):
        if abs(omega1[i]) > limit and not in_peak:
            in_peak = True
            counter += 1
            for j in range(index2, len(omega2), 1):
                if abs(omega2[j]) > limit:
                    same_time.append((i,j))
                    index2 = j
                    break
            
        elif in_peak and abs(omega1[i]) < limit:
            in_peak = False
            for j in range(index2, len(omega2), 1):
                if abs(omega2[j]) < limit:
                    same_time.append((i,j))
                    index2 = j
                    break
            if counter == n_peaks:
                break
    if counter < n_peaks:
        print("Only ", counter, " peaks in data")
    else:
        return same_time

In [5]:
def find_points2(omegaX, omegaZ, omega2, n_peaks, limit):
    counter = 0
    in_peak = False
    same_time = []
    index1 = 0 
    indexX = 0
    indexZ = 0

    for i in range(len(omega2)):
        if abs(omega2[i]) > limit and not in_peak:
            in_peak = True
            counter += 1
            for j in range(index1, len(omegaX), 1):
                if abs(omegaX[j]) > limit:
                    indexX = j
                    break
            for k in range(index1, len(omegaZ), 1):
                if abs(omegaZ[k]) > limit:
                    indexZ = k
                    break
            same_time.append((i, min(indexX, indexZ)))
            index1 = min(indexX, indexZ)
            
        elif in_peak and abs(omega2[i]) < limit:
            in_peak = False
            if indexX < indexZ:
                for j in range(index1, len(omegaX), 1):
                    if abs(omegaX[j]) < limit:
                        same_time.append((i,j))
                        index1 = j
                        break
            if indexX > indexZ:
                for k in range(index1, len(omegaZ), 1):
                    if abs(omegaZ[k]) < limit:
                        same_time.append((i,k))
                        index1 = k
                        break
            if counter == n_peaks:
                break
                
    if counter < n_peaks:
        print("Only ", counter, " peaks in data")
    else:
        return same_time

In [6]:
def concatenate_axes(time1, time2, fi1, fi2, omega1, omega2):
    #time_1_2ibl = a1*time_1 + b1
    #time_2_2ibl = a2*time_2 + b2

    time12    = np.empty(time1.size + time2.size, dtype=time1.dtype)
    fi1_12    = np.empty(time1.size + time2.size, dtype=time1.dtype) 
    fi2_12    = np.empty(time1.size + time2.size, dtype=time1.dtype)
    omega1_12 = np.empty(time1.size + time2.size, dtype=time1.dtype)
    omega2_12 = np.empty(time1.size + time2.size, dtype=time1.dtype)
    i, j, k = 0, 0, 0

    while i < time1.size and j < time2.size:
        if time1[i] < time2[j]:
            time12[k] = time1[i]
            fi1_12[k] = fi1[i]
            omega1_12[k] = omega1[i]
            fi2_12[k] = 0 if k==0 else fi2_12[k-1]
            omega2_12[k] = 0 if k==0 else omega2_12[k-1]
            i += 1
        else:
            time12[k] = time2[j]
            fi2_12[k] = fi2[j]
            omega2_12[k] = omega2[j]
            fi1_12[k] = 0 if k==0 else fi1_12[k-1]
            omega1_12[k] = 0 if k==0 else omega1_12[k-1]
            j += 1
        k += 1

    # Добавляем оставшиеся элементы
    while i < time1.size:
        time12[k] = time1[i]
        fi1_12[k] = fi1[i]
        omega1_12[k] = omega1[i]
        fi2_12[k] = fi2_12[k-1]
        omega2_12[k] = omega2_12[k-1]
        i += 1
        k += 1

    while j < time2.size:
        time12[k] = time2[j]
        fi2_12[k] = fi2[j]
        omega2_12[k] = omega2[j]
        fi1_12[k] = fi1_12[k-1]
        omega1_12[k] = omega1_12[k-1]
        j += 1
        k += 1
    
    return time12, fi1_12, fi2_12, omega1_12, omega2_12

In [7]:
def output_reference(fi1_12, fi2_12, omega1_12, omega2_12, time12, begin_koord, etalon_name, Omega_earth=np.array([0, 12.46, 8.42]), g=np.array([0, 9.8153, 0])):    
    OmegaX = np.array([])
    OmegaY = np.array([])
    OmegaZ = np.array([])

    M12 = np.array([])
    ex = begin_koord[:, 0]
    ey = begin_koord[:, 1]
    ez = begin_koord[:, 2]
    fi10, fi1  = 0, 0
    fi20, fi2  = 0, 0

    file = open(etalon_name, 'w')
    file.write("{:8}".format("time") + "{:17}".format("omega_x,gph") + "{:17}".format("omega_y,gph") + "{:17}".format("omega_z,gph") + "{:12}".format("w_x,gph") + "{:12}".format("w_y,gph") + "{:12}".format("w_z,gph") + "{:17}".format("omegaxe,gph") + "{:17}".format("omegaye,gph") +"{:17}".format("omegaze,gph") + 'ex     ' + 'ey     ' + 'ez     '+'\n')
    for i in range(1, len(time12)//10 +1):
        fi10, fi1 = fi1_12[i-1], fi1_12[i]
        fi20, fi2 = fi2_12[i-1], fi2_12[i]
        dfi1 = math.radians(fi1-fi10)
        dfi2 = math.radians(fi2-fi20)
        
        M2 = np.array([[1,      0,             0      ],      # матрица поворота, [0, np.cos(dfi2), -np.sin(dfi2)], если ось 2 сонаправлена с X
                       [0, np.cos(dfi2),  np.sin(dfi2)],      # наоборот если противоположное направление
                       [0, -np.sin(dfi2),  np.cos(dfi2)]])
        ex = M2 @ ex
        ey = M2 @ ey
        ez = M2 @ ez
        
        y, z = -ey[1], -ey[2]  # - потому что ось 1 не сонаправлена с y
        M12 = np.array([[np.cos(dfi1),                                 -(y*np.sin(dfi2)+z*np.cos(dfi2))*np.sin(dfi1),                                    (y*np.cos(dfi2)-z*np.sin(dfi2))*np.sin(dfi1)],
                        [(y*np.sin(dfi2)+z*np.cos(dfi2))*np.sin(dfi1), np.cos(dfi1)+(1-np.cos(dfi1))*(y*np.cos(dfi2)-z*np.sin(dfi2))**2,                 (1-np.cos(dfi1))*(y*np.cos(dfi2)-z*np.sin(dfi2))*(y*np.sin(dfi2)+z*np.cos(dfi2))], 
                        [(z*np.sin(dfi2)-y*np.cos(dfi2))*np.sin(dfi1), (1-np.cos(dfi1))*(y*np.sin(dfi2)+z*np.cos(dfi2))*(y*np.cos(dfi2)-z*np.sin(dfi2)), np.cos(dfi1)+(1-np.cos(dfi1))*(y*np.sin(dfi2)+z*np.cos(dfi2))**2]])
        ex = M12 @ ex
        ey = M12 @ ey
        ez = M12 @ ez
 
        omegax = np.dot(ex, Omega_earth) - omega2_12[i]*ex[0]  #- потому что ось 2 не сонаправлена с X
        omegay = np.dot(ey, Omega_earth) - omega1_12[i]  #- потому что ось 1 не сонаправлена с y
        omegaz = np.dot(ez, Omega_earth) - omega2_12[i]*ez[0]  #- потому что ось 2 не сонаправлена с X

        file.write("{:8.3f}".format(time12[i]) + "{:17.8f}".format(omegax) + "{:17.8f}".format(omegay) + "{:17.8f}".format(omegaz) + "{:12.8f}".format(np.dot(ex, g))  + "{:12.8f}".format(np.dot(ey, g))  + "{:12.8f}".format(np.dot(ez, g)) + '\n')
    file.close()

In [8]:
def build_reference(data_ibl, data_1, data_2, npeaks=10, limit1=500, limit2=600, begin_koord=np.array([[1,0,0],[0,1,0],[0,0,1]]), etalon_name="etalon.txt"):
    time_ibl = data_ibl[:, 0]
    omegaX_ibl = data_ibl[:, 5]
    omegaY_ibl = data_ibl[:, 6] * (-1) #-1 потому что оси несонаправлены
    omegaZ_ibl = data_ibl[:, 7]
                    
    time_1 = data_1[:, 0]
    fi_1 = data_1[:, 1]
    omega_1 = data_1[:, 2]*3600
                    
    time_2 = data_2[:, 0]
    fi_2 = data_2[:, 1]
    omega_2 = data_2[:, 2]*3600
                    
    points1 = find_points1(omegaY_ibl, omega_1, npeaks, limit1)
    points2 = find_points2(omegaX_ibl, omegaZ_ibl, omega_2, npeaks, limit2)
                    
    timeibl1 = []
    time1 = []
    for i in range(len(points1)):
        timeibl1.append(time_ibl[points1[i][0]])
        time1.append(time_1[points1[i][1]])
    
    timeibl2 = []
    time2 = []
    for i in range(len(points2)):
        timeibl2.append(time_ibl[points2[i][1]])
        time2.append(time_2[points2[i][0]])
    
    a1, b1 = np.polyfit(time1, timeibl1, 1)
    a2, b2 = np.polyfit(time2, timeibl2, 1)
                    
    time_1_2ibl = a1*time_1 + b1
    time_2_2ibl = a2*time_2 + b2
    
    time12, fi1_12, fi2_12, omega1_12, omega2_12 = concatenate_axes(time_1_2ibl, time_2_2ibl, fi_1, fi_2, omega_1, omega_2)
    
    output_reference(fi1_12, fi2_12, omega1_12[:], omega2_12[:], time12[:], begin_koord, etalon_name)

In [9]:
def build_data(etalon, data_ibl):
    data = np.zeros((etalon.shape[0], data_ibl.shape[1]))
    index = 0
    for i, time in enumerate(etalon[:, 0]):
        index = int(time//0.005 + int((time%0.005) > 0.0025))   # не универсально, зависит от Тси
        data[i, :] = data_ibl[index, :] 
    return data

In [10]:
data_ibl = np.load("20240204_1426.npy")
data_1 = np.load("05022024_1.npy")
data_2 = np.load("05022024_2.npy")

In [11]:
build_reference(data_ibl, data_1, data_2)

In [12]:
etalon = np.genfromtxt("etalon.txt", skip_header=1)
np.save("etalon.npy", etalon)

In [13]:
data = build_data(etalon, data_ibl)[:, [0,2,3,4,8,9,10,14,15,16,17,18,19,20]]
np.savetxt("data.txt", data, delimiter=' ')
np.save("data.npy", data)